In [ ]:
!pip install pandas openpyxl reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet
from google.colab import files


In [ ]:
# Step 1: Upload the Excel file
print("Please upload your Excel file (student_scores.xlsx):")
uploaded = files.upload()


Please upload your Excel file (student_scores.xlsx):


Saving student_scores.csv to student_scores.csv


In [ ]:
if uploaded:
    file_name = list(uploaded.keys())[0]
    print(f"Uploaded file: {file_name}")

    try:
        # Step 2: Read the CSV file
        df = pd.read_csv(file_name)
        print("CSV file successfully read.")
        print("Preview of the data:")
        print(df.head())  # Display the first few rows for debugging
    except Exception as e:
        print(f"Error reading the CSV file: {e}")
        df = None
else:
    print("No file uploaded. Exiting.")
    df = None

if df is not None:
    # Step 3: Check for Missing Data
    if df.isnull().values.any():
        print("Warning: Missing data found. Dropping rows with missing data.")
        df = df.dropna()

    # Ensure the expected columns exist
    required_columns = {'Student ID', 'Name', 'Subject', 'Subject Score'}
    if not required_columns.issubset(df.columns):
        print(f"Error: Missing required columns. Expected: {required_columns}")
    else:
        # Step 4: Group Data by Student
        try:
            student_grouped = df.groupby(['Student ID', 'Name']).agg(
                total_score=('Subject Score', 'sum'),
                average_score=('Subject Score', 'mean')
            ).reset_index()
            print("Data grouped successfully. Here's a preview:")
            print(student_grouped.head())
        except Exception as e:
            print(f"Error during grouping: {e}")
            student_grouped = None

        if student_grouped is not None:
            # Step 5: Generate PDF Report Cards
            styles = getSampleStyleSheet()

            def generate_pdf(student_id, student_name, total_score, average_score, subjects_scores):
                pdf_filename = f"report_card_{student_id}.pdf"
                document = SimpleDocTemplate(pdf_filename, pagesize=letter)
                content = []

                # Add student details
                content.append(Paragraph(f"Report Card for {student_name}", styles['Title']))
                content.append(Paragraph(f"Total Score: {total_score}", styles['Normal']))
                content.append(Paragraph(f"Average Score: {average_score:.2f}", styles['Normal']))

                # Create table for subject-wise scores
                table_data = [['Subject', 'Score']]
                for subject, score in subjects_scores.items():
                    table_data.append([subject, score])

                table = Table(table_data)
                table.setStyle(TableStyle([
                    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                    ('GRID', (0, 0), (-1, -1), 1, colors.black),
                ]))

                content.append(table)


Uploaded file: student_scores.csv
CSV file successfully read.
Preview of the data:
   Student ID       Name   Subject  Subject Score
0         101   John Doe     Maths             90
1         101   John Doe  English              91
2         101   John Doe   Science             92
3         101  Ane Smith     Maths             88
4         102  Ane Smith  English              89
Data grouped successfully. Here's a preview:
   Student ID        Name  total_score  average_score
0         101   Ane Smith           88           88.0
1         101    John Doe          273           91.0
2         102   Ane Smith           89           89.0
3         106   Ane Smith           84           84.0
4         107  Sam Wilson           86           86.0


In [ ]:
# Build the document (Generate PDF)
    document.build(content)
    print(f"Generated PDF: {pdf_filename}")

In [ ]:
# Generate a report card for each student
            for _, student_data in student_grouped.iterrows():
                student_id = student_data['Student ID']
                student_name = student_data['Name']
                total_score = student_data['total_score']
                average_score = student_data['average_score']

                subjects_scores = df[df['Student ID'] == student_id][['Subject', 'Subject Score']].set_index('Subject').to_dict()['Subject Score']
                generate_pdf(student_id, student_name, total_score, average_score, subjects_scores)

            # Step 6: Zip and Download PDFs
            import shutil
            shutil.make_archive("report_cards", 'zip', "./")
            print("All PDFs generated. Download the ZIP file:")
            files.download("report_cards.zip")
else:
    print("Exiting due to missing or invalid data.")

**Explanation:**

1.**Reading and Processing the Excel File:**
The script uses pd.read_excel() to read the data.
It checks for missing data using isnull().values.any(),  and drops rows with missing values using dropna().

**2.Calculating Total and Average Scores:**
The groupby() function is used to group data by Student ID and Name. Then, the agg() method calculates the total and average scores for each student.

**3.Generating PDFs:**
ReportLab is used to generate a PDF for each student. A table is created to display subject-wise scores.
The student’s name, total score, and average score are displayed at the top.

**4.Saving the PDFs:**
The report card for each student is saved as report_card_<StudentID>.pdf.

**5.Libraries Required:**
pandas for data manipulation
reportlab for generating PDFs

**6.Error Handling:**
The script handles potential errors in reading the Excel file and missing data by dropping rows with NaN values.
It generates a warning if any missing values are found and skips them while processing.
This script should work for your task of generating PDF report cards for each student from the given Excel file.